In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
from torchsummary import summary 
import onnx 
import onnxruntime as ort
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error
import random
import os
import datetime
import matplotlib.pyplot as plt

In [2]:
import os, sys
REPO = "/home/deepaksr/project/Project_files_2/U-2-Net"    
os.chdir(REPO)
sys.path.insert(0, REPO)

In [3]:
from model import U2NET, U2NETP

In [4]:
class Saliency_IR_v1(nn.Module):
    def __init__(self):
        super(Saliency_IR_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
class Saliency_IR_v2(nn.Module):
    def __init__(self):
        super(Saliency_IR_v2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(8, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## d1

In [17]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d1_20250511_201145/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d1_fixed_bce_20250514_091441/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v2_d1_fixed_bce_20250514_105129/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(1,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Your Saliency_IR_v1
model_v1 = Saliency_IR_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Your Saliency_IR_v2
model_v2 = Saliency_IR_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3252004/896093851.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_325200

In [18]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/IR/simulated_dataset"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_d1.pt")
print("Saved u2netp_finetuned_d1.pt")

# 2) Script Saliency_IR_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_ir_v1_d1.pt")
print("Saved saliency_ir_v1_d1.pt")

# 3) Script Saliency_IR_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_ir_v2_d1.pt")
print("Saved saliency_ir_v2_d1.pt")


Saved u2netp_finetuned_d1.pt
Saved saliency_ir_v1_d1.pt
Saved saliency_ir_v2_d1.pt


## d2

In [21]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d2_20250511_210042/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d2_fixed_bce_20250515_131724/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v2_d2_fixed_bce_20250515_152205/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(1,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Saliency_IR_v1
model_v1 = Saliency_IR_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Saliency_IR_v2
model_v2 = Saliency_IR_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3252004/3471408118.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_32520

In [22]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/IR/paper_dataset"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_d2.pt")
print("Saved u2netp_finetuned_d2.pt")

# 2) Script Saliency_IR_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_ir_v1_d2.pt")
print("Saved saliency_ir_v1_d2.pt")

# 3) Script Saliency_IR_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_ir_v2_d2.pt")
print("Saved saliency_ir_v2_d2.pt")


Saved u2netp_finetuned_d2.pt
Saved saliency_ir_v1_d2.pt
Saved saliency_ir_v2_d2.pt


## d3

In [23]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d3_20250511_212158/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d3_fixed_bce_20250517_132705/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v2_d3_fixed_bce_20250517_221851/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(1,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Saliency_IR_v1
model_v1 = Saliency_IR_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Saliency_IR_v2
model_v2 = Saliency_IR_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3252004/1736575951.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_32520

In [24]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/IR/actual_ir"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_d3.pt")
print("Saved u2netp_finetuned_d3.pt")

# 2) Script Saliency_IR_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_ir_v1_d3.pt")
print("Saved saliency_ir_v1_d3.pt")

# 3) Script Saliency_IR_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_ir_v2_d3.pt")
print("Saved saliency_ir_v2_d3.pt")


Saved u2netp_finetuned_d3.pt
Saved saliency_ir_v1_d3.pt
Saved saliency_ir_v2_d3.pt


## d4

In [25]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d4_20250512_010041/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d4_fixed_bce_20250515_164139/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v2_d4_fixed_bce_20250515_171405/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(1,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Saliency_IR_v1
model_v1 = Saliency_IR_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Saliency_IR_v2
model_v2 = Saliency_IR_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3252004/3445202001.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_32520

In [26]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/IR/vehicle_ir"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_d4.pt")
print("Saved u2netp_finetuned_d4.pt")

# 2) Script Saliency_IR_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_ir_v1_d4.pt")
print("Saved saliency_ir_v1_d4.pt")

# 3) Script Saliency_IR_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_ir_v2_d4.pt")
print("Saved saliency_ir_v2_d4.pt")


Saved u2netp_finetuned_d4.pt
Saved saliency_ir_v1_d4.pt
Saved saliency_ir_v2_d4.pt


## d5

In [27]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d5_20250512_011730/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d5_fixed_bce_20250515_173850/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v2_d5_fixed_bce_20250515_181709/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(1,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Saliency_IR_v1
model_v1 = Saliency_IR_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Saliency_IR_v2
model_v2 = Saliency_IR_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3252004/4018318598.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_32520

In [28]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/IR/hit-uav"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_d5.pt")
print("Saved u2netp_finetuned_d5.pt")

# 2) Script Saliency_IR_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_ir_v1_d5.pt")
print("Saved saliency_ir_v1_d5.pt")

# 3) Script Saliency_IR_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_ir_v2_d5.pt")
print("Saved saliency_ir_v2_d5.pt")


Saved u2netp_finetuned_d5.pt
Saved saliency_ir_v1_d5.pt
Saved saliency_ir_v2_d5.pt


## Combined

In [ ]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d6_combined_20250514_205349/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d6_combined_fixed_bce_20250518_182701/best_model.pth"
pth_v2 = ""

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(1,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Saliency_IR_v1
model_v1 = Saliency_IR_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Saliency_IR_v2
model_v2 = Saliency_IR_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")
